In [1]:
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm
from pathlib import Path
import pycolmap
import os

from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_exhaustive, pairs_from_retrieval
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

In [2]:
name = 'town'
images = Path(f'data/{name}') # 图片的路径
db_images = images / 'db'
query_images = images / 'query'
outputs = Path(f'outputs/town_retrieval/') # 输出的路径
sfm_pairs = outputs / 'pairs-netvlad.txt' # sfm的配对文件
loc_pairs = outputs / 'pairs-loc.txt' # loc的配对文件
sfm_dir = outputs / 'sfm'

feature_conf = extract_features.confs['disk']
matcher_conf = match_features.confs['disk+lightglue']

retrieval_conf = extract_features.confs['netvlad']  # global descriptors

retrieval_path = outputs / 'global-feats-netvald.h5'
feature_path = outputs / 'features-disk.h5'
match_path = outputs / 'feature-disk_matches-disk-lightglue_pairs-netvlad.h5'

In [4]:
db_image_list = [p.relative_to(images).as_posix() for p in (db_images).iterdir()]
print(db_image_list)
print(len(db_image_list))

['db/img58.jpg', 'db/img69.jpg', 'db/img37.jpg', 'db/img33.jpg', 'db/img49.jpg', 'db/img38.jpg', 'db/img1.jpg', 'db/img30.jpg', 'db/img71.jpg', 'db/img53.jpg', 'db/img27.jpg', 'db/img57.jpg', 'db/img36.jpg', 'db/img68.jpg', 'db/img0.jpg', 'db/img67.jpg', 'db/img29.jpg', 'db/img5.jpg', 'db/img6.jpg', 'db/img52.jpg', 'db/img24.jpg', 'db/img47.jpg', 'db/img51.jpg', 'db/img39.jpg', 'db/img65.jpg', 'db/img50.jpg', 'db/img21.jpg', 'db/img70.jpg', 'db/img11.jpg', 'db/img12.jpg', 'db/img45.jpg', 'db/img64.jpg', 'db/img13.jpg', 'db/img10.jpg', 'db/img23.jpg', 'db/img66.jpg', 'db/img42.jpg', 'db/img44.jpg', 'db/img62.jpg', 'db/img61.jpg', 'db/img34.jpg', 'db/img14.jpg', 'db/img63.jpg', 'db/img22.jpg', 'db/img25.jpg', 'db/img3.jpg', 'db/img15.jpg', 'db/img2.jpg', 'db/img19.jpg', 'db/img46.jpg', 'db/img16.jpg', 'db/img9.jpg', 'db/img55.jpg', 'db/img35.jpg', 'db/img54.jpg', 'db/img26.jpg', 'db/img28.jpg', 'db/img31.jpg', 'db/img41.jpg', 'db/img8.jpg', 'db/img59.jpg', 'db/img60.jpg', 'db/img4.jpg', 

In [5]:
retrieval_path = extract_features.main(
    retrieval_conf, 
    images, 
    outputs, 
    image_list=db_image_list, 
    overwrite=False # 表示在h5内提取过的特征是否重新提取
    )
pairs_from_retrieval.main(retrieval_path, sfm_pairs, num_matched=5)

[2024/12/22 12:49:06 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}


  0%|          | 0/66 [00:00<?, ?it/s]

[2024/12/22 12:49:16 hloc INFO] Finished exporting features.
[2024/12/22 12:49:16 hloc INFO] Extracting image pairs from a retrieval database.
[2024/12/22 12:49:16 hloc INFO] Found 330 pairs.


True


In [6]:
# get local descriptors
feature_path = extract_features.main(feature_conf, images, outputs, image_list=db_image_list, overwrite=True)

[2024/12/22 12:49:24 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 5000, 'name': 'disk'},
 'output': 'feats-disk',
 'preprocessing': {'grayscale': False, 'resize_max': 800}}


  0%|          | 0/66 [00:00<?, ?it/s]

[2024/12/22 12:49:27 hloc INFO] Finished exporting features.


In [7]:
match_path = match_features.main(matcher_conf, sfm_pairs, feature_conf["output"], outputs)

[2024/12/22 12:49:29 hloc INFO] Matching local features with configuration:
{'model': {'features': 'disk', 'name': 'lightglue'},
 'output': 'matches-disk-lightglue'}
/home/ps/.conda/envs/hloc/lib/python3.8/site-packages/lightglue/lightglue.py:143: UserWarning: FlashAttention is not available. For optimal speed, consider installing torch >= 2.0 or flash-attn.
  self.inner_attn = Attention(flash)


  0%|          | 0/199 [00:00<?, ?it/s]

[2024/12/22 12:49:50 hloc INFO] Finished exporting matches.


In [8]:
model = reconstruction.main(sfm_dir, images, sfm_pairs, feature_path, match_path, image_list=db_image_list)

[2024/12/22 12:49:53 hloc INFO] Creating an empty database...
[2024/12/22 12:49:53 hloc INFO] Importing images into the database...
[2024/12/22 12:49:53 hloc INFO] Importing features into the database...


  0%|          | 0/66 [00:00<?, ?it/s]

[2024/12/22 12:49:53 hloc INFO] Importing matches into the database...


  0%|          | 0/330 [00:00<?, ?it/s]

[2024/12/22 12:49:53 hloc INFO] Performing geometric verification of the matches...
I20241222 12:49:53.895134 140425746122496 misc.cc:198] 
Feature matching
I20241222 12:49:53.895303 140424739518208 sift.cc:1457] Creating SIFT CPU feature matcher
I20241222 12:49:53.895315 140425037842176 sift.cc:1457] Creating SIFT CPU feature matcher
I20241222 12:49:53.895330 140425138521856 sift.cc:1457] Creating SIFT CPU feature matcher
I20241222 12:49:53.895349 140425704175360 sift.cc:1457] Creating SIFT CPU feature matcher
I20241222 12:49:53.895375 140425662228224 sift.cc:1457] Creating SIFT CPU feature matcher
I20241222 12:49:53.895389 140425620281088 sift.cc:1457] Creating SIFT CPU feature matcher
I20241222 12:49:53.895393 140425578333952 sift.cc:1457] Creating SIFT CPU feature matcher
I20241222 12:49:53.895411 140425264363264 sift.cc:1457] Creating SIFT CPU feature matcher
I20241222 12:49:53.895432 140425222416128 sift.cc:1457] Creating SIFT CPU feature matcher
I20241222 12:49:53.895435 1404251